In [ ]:
%load_ext autoreload
import os

os.chdir("../")

In [ ]:
%autoreload
from src.iris_manager import IRISManager
from src.utils import get_date_chunks
import datetime as dt
import pandas as pd
from tqdm import tqdm
import plotly.graph_objects as go
import logging
import os
from glob import glob
from PIL import Image
import numpy as np

logger = logging.getLogger(__name__)

In [ ]:
BMUS = [
    ("T_BLHLB-1", "N. Scotland"),
    ("E_JAMBB-1", "C. Scotland"),
    ("T_NTRVB-1", "S. Scotland"),
    ("T_OCHLB-1", "Midlands"),
    ("T_LARKB-1", "SW England"),
    ("E_FARNB-1", "S. England"),
]

end = dt.datetime(2025, 8, 31, tzinfo=dt.timezone.utc)
start = dt.datetime(2025, 1, 1, tzinfo=dt.timezone.utc)
date_ranges = get_date_chunks(start_date=start, end_date=end)

client = IRISManager()

In [ ]:
dfs = {}

for bmu_tuple in tqdm(BMUS):
    _dfs = []
    for start_date, end_date in tqdm(date_ranges):
        _df = client.get_physical_data(
            bmu=bmu_tuple[0],
            start_date=start_date,
            end_date=end_date,
            datasets=["PN"],
        )
        _dfs.append(_df)

    df = pd.concat(_dfs).reset_index(drop=True)

    if df.empty:
        logger.warning(f"No records found for BMU: {bmu_tuple[0]}")
        break

    df = df.drop(["dataset", "national_grid_bm_unit"], axis=1)
    df = df.sort_values("time_from", ascending=True)

    # Resample start time to 30min and average the level to account for non-HH FPNs
    # For full precision, a time weighted average should be taken here however
    # the vast majority of FPNs submitted will be HH so this does not totally break accuracy
    df = (
        df.resample(rule="30min", on="time_from")
        .agg(
            {
                "settlement_date": "first",
                "settlement_period": "first",
                "level_from": "mean",
            }
        )
        .reset_index()
    )

    counts = df["settlement_date"].value_counts()
    df = df[df["settlement_date"].map(counts) > 47].reset_index(drop=True)

    df["HH_STRING"] = df["time_from"].dt.strftime("%H:%M")
    dfs[bmu_tuple[0]] = df

GENERATION_TIME = dt.datetime.now()


In [ ]:
for bmu_tuple in tqdm(BMUS):
    _df = dfs[bmu_tuple[0]]
    fig = go.Figure(
        data=go.Heatmap(
            z=_df["level_from"],
            x=_df["settlement_date"],
            y=_df["HH_STRING"],
            colorscale="RdBu_r",
            colorbar=dict(
                lenmode="fraction",
                len=0.75,
                dtick=10,
                thickness=20,
                title=dict(text="FPN (MW)", side="right"),
            ),
            zmin=_df["level_from"].min().round(),
            zmax=_df["level_from"].max().round(),
            xgap=0.5,
            ygap=0.5,
        )
    )

    fig.update_layout(
        template="simple_white",
        title=dict(
            text=f"{bmu_tuple[0]} ({bmu_tuple[1]}) - Half Hourly Final Physical Notication (FPN)",
            x=0.5,
            subtitle=dict(
                text=f"Start: {start.strftime('%Y-%m-%d')} - End: {end.strftime('%Y-%m-%d')} | Generated at {GENERATION_TIME.strftime('%Y-%m-%d %H:%M %Z')}",
                font=dict(color="black", size=12),
            ),
        ),
        yaxis_nticks=24,
        width=2000,
        height=600,
        plot_bgcolor="black",
        yaxis=dict(
            title=dict(text="Start Time (UTC)"),
            autorange="reversed",
            showline=True,
            showgrid=False,
        ),
        xaxis=dict(
            title=dict(text="Settlement Date"),
            showline=True,
            showgrid=False,
            mirror=True,
            type="date",
            dtick="M1",  # Set ticks to the first of every 1 month
            tickformat="%b-%d-%Y",
        ),
    )

    display(fig)
    fig.write_image(
        f"./notebooks/figures/{bmu_tuple[0]}_FPN_HEATMAP_{start.strftime('%Y_%m_%d')}_{end.strftime('%Y_%m_%d')}.png",
        width=2000,
        height=600,
    )
    del fig

In [ ]:
# Create horizontally combined chart # https://stackoverflow.com/a/78580732

# Use this for automatic file discovery
# images_folder = "./notebooks/figures/"
# list_im = glob(os.path.join(images_folder, "*.png"))

# Use this to specifically define order
list_im = [
    "./notebooks/figures/T_BLHLB-1_FPN_HEATMAP_2025_01_01_2025_08_31.png",
    "./notebooks/figures/E_JAMBB-1_FPN_HEATMAP_2025_01_01_2025_08_31.png",
    "./notebooks/figures/T_NTRVB-1_FPN_HEATMAP_2025_01_01_2025_08_31.png",
    "./notebooks/figures/T_OCHLB-1_FPN_HEATMAP_2025_01_01_2025_08_31.png",
    "./notebooks/figures/T_LARKB-1_FPN_HEATMAP_2025_01_01_2025_08_31.png",
    "./notebooks/figures/E_FARNB-1_FPN_HEATMAP_2025_01_01_2025_08_31.png",
]

imgs = [Image.open(i) for i in list_im]
min_shape = sorted([(np.sum(i.size), i.size) for i in imgs])[0][1]
imgs_comb = np.vstack([i.resize(min_shape) for i in imgs])

imgs_comb = Image.fromarray(imgs_comb)
imgs_comb.save("./notebooks/MULTI_BMU_FPN_HEATMAP.png")